<a href="https://colab.research.google.com/github/JBenedictM/501-A4-TF/blob/master/501_A4_p3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

In [26]:
from google.colab import drive
drive.mount('/content/drive')
data_path = "/content/drive/My Drive/heart.csv"
train_path = "/content/drive/My Drive/heart_train.csv"
test_path = "/content/drive/My Drive/heart_test.csv"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Create Train and Test csv files
Splits Heart.csv into 80% training data and 20% testing data

In [0]:
import pandas as pd
from sklearn.model_selection import train_test_split
import sys

data = pd.read_csv(data_path)
y_chd = data.chd
X = data.drop("chd", axis=1)
X = X.drop("row.names", axis=1)
#data = data.drop("", axis=1)
#print(X)

x_train, x_test, y_train, y_test = train_test_split(data, y_chd, test_size=0.2)

#print(x_train.shape)
#print(x_train)

#print(x_test.shape)
#print(x_test)

x_train.to_csv("/content/drive/My Drive/heart_train.csv", encoding="utf-8", sep='\t', index=False)
x_test.to_csv("/content/drive/My Drive/heart_test.csv", encoding="utf-8", sep='\t', index=False)

# Load and Process Data

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals
import functools

import numpy as np
import tensorflow as tf
import pandas as pd

# load csv data
def get_dataset(file_path, **kwargs):
    dataset = tf.data.experimental.make_csv_dataset(
        data_path,
        label_name=PREDICT_COLUMN,
        batch_size=10,
        na_value="?",
        num_epochs=1,
        ignore_errors=True,
        **kwargs)
    
    return dataset

def show_batch(dataset):
    for batch, label in dataset.take(1):
        for key, value in batch.items():
            print("{:20s}: {}".format(key, value.numpy()))

def pack(features, label):
    return tf.stack(list(features.values()), axis=-1), label



In [0]:
class PackNumericFeatures(object):
    def __init__(self, names):
        self.names = names

      
    def __call__(self, features, labels):
        numeric_features = [features.pop(name) for name in self.names]
        numeric_features = [tf.cast(feat, tf.float32) for feat in numeric_features]
        numeric_features = tf.stack(numeric_features, axis=-1)
        features["numeric"] = numeric_features

        return features, labels


In [46]:
# instaniate training and testing data
PREDICT_COLUMN = "chd"
PREDICT_LABELs = [0, 1]


NAME_COLUMN = "row.names"
SELECT_COLUMNS = [NAME_COLUMN] + ["sbp", "tobacco", "ldl", "adiposity", "famhist", "typea", "obesity", "alcohol", "age"] + [PREDICT_COLUMN]
NUMERIC_COLUMNS = [NAME_COLUMN] + ["sbp", "tobacco", "ldl", "adiposity", "typea", "obesity", "alcohol", "age"]
SELECT_COLUMNS_NUM = NUMERIC_COLUMNS + [PREDICT_COLUMN]
DEFAULT_NUM = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

raw_train_data = get_dataset(train_path, select_columns=SELECT_COLUMNS)
raw_test_data = get_dataset(test_path, select_columns=SELECT_COLUMNS)

#show_batch(raw_train_data)
#print()
#show_batch(raw_test_data)

temp_train_numeric = get_dataset(train_path, select_columns=SELECT_COLUMNS_NUM, column_defaults=DEFAULT_NUM)
#show_batch(temp_train_numeric)

example_batch, labels_batch = next(iter(temp_train_numeric))

packed_dataset = temp_train_numeric.map(pack)

#for features, label in packed_dataset.take(1):
 #   print(features.numpy())
  #  print()
   # print(label.numpy())

packed_train_data = raw_train_data.map(PackNumericFeatures(NUMERIC_COLUMNS))
packed_test_data = raw_test_data.map(PackNumericFeatures(NUMERIC_COLUMNS))
show_batch(packed_train_data)

# enumerate data


famhist             : [b'Absent' b'Absent' b'Present' b'Absent' b'Present' b'Absent' b'Absent'
 b'Absent' b'Present' b'Absent']
numeric             : [[ 36.   122.     4.26   4.44  13.04  57.    19.49  48.99  28.  ]
 [ 70.   122.     0.     3.37  16.1   67.    21.06   0.    32.  ]
 [449.   130.     0.     1.88  12.51  52.    20.28   0.    17.  ]
 [313.   114.     9.6    2.51  29.18  49.    25.67  40.63  46.  ]
 [370.   126.     3.4    4.87  15.16  65.    22.01  11.11  38.  ]
 [178.   120.     0.     2.42  16.66  46.    20.16   0.    17.  ]
 [ 93.   143.     0.46   2.4   22.87  62.    29.17  15.43  29.  ]
 [ 84.   148.    12.2    3.79  34.15  57.    26.38  14.4   57.  ]
 [169.   126.     0.54   4.39  21.13  45.    25.99   0.    25.  ]
 [ 34.   126.     8.75   6.53  34.02  49.    30.25   0.    41.  ]]
